# Optimization of liquid transfer paramters of viscous liquids guided by Baysesian Optimization

## 1. Introduction

This jupyter notebook contains the information required to perform Multi-Bayesian optimization (MOBO) of liquid handling parameters of pipetting robots according to the protocol described in **publication add link** . The notebook is divided in the following sections

1. **Imports**: This section contains the relevant packages required to perform MOBO

2. **BO Liquid Tansfer class**: This section defines the BO_LiqTransfer class that contains the functions required to implement the MOBO through semi-automataed and fully automated methods. The semi-autmated method is prefered when the method to control the liquid transfering robot cannot be implemented with external libraries such as Torch. the fully-automated implementation in this notebook is defined to control a rLine1000 Sartorious pipette coupled to a M1 Dobot Scara robotic arm controlled by control-lab-ly python package.

3. **Semi-automated implemntation**: This section contains the code to obtain  suggestions for liquid handling parameters where the robotic platform performing the transfers is controlled in a separte script. This is the method that was used in **publication** to optimize the liquid transfer paramters of a Opentrons OT2 robot.

4. **Fully-automated implemetnation**: This section contains the code to obtain  suggestions for liquid handling parameters where the robotic platform performing the transfers is a rLine1000 Sartorious pipette coupled to a M1 Dobot Scara robotic arm controlled by control-lab-ly python package. This is the method that was used in **publication** to optimize the liquid transfer paramters of a rLine1000 pipette in the fully automated optimization experiments.

## 1. Imports

In [ ]:
# basic dependencies

# basic dependencies

import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

import pandas as pd
import time


## To process mass data and fit sigmid curves in automated initialization experiments
from scipy import signal
from scipy.optimize import curve_fit

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly

torch.set_printoptions(precision=3)

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement

# utilities
from botorch.optim.optimize import optimize_acqf
from botorch.sampling import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

from gpytorch.constraints import GreaterThan
from torch.optim import SGD
from gpytorch.mlls import ExactMarginalLogLikelihood

import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)



## 2. Bayesian optimization liquid transfer class

In [ ]:
class BO_LiqTransfer:
    """
    BO_LiqTransfer provides methods to perform Bayesian Optimization of liquid handling paramters of pipetting robots to transfer viscous liquids 

    ### Constructor
    Args:
        `liquid_name` (str): Name of the target liquid that requires liquid handling parameter optimization 
        `density` (float, Optional): Density of the target liquid.
    
    ### Attributes
    - 'features' (list): Column names from _data othat will be used as features for the optimization
    - 'objectives' (list): Column names from _data othat will be used as objectives for the optimization
    - 'bmax' (float): Factor that scales initial flow rate rate to maximum value 
    - 'bmin' (float): Factor that scales initial flow rate rate to minimum value 
    - 'mean_volumes' (list): List of volumes that will be used for optimization 
    - 'platform' (device): Object pointing to automated equipment used during optimization
    
    ### Properties (with setter)
    - '_data' (DataFrame): Dataframe containing the data relevant to the experiments
    - '_param_dict' (dict): Dictionary containing the liquid handling paramters of the autoamted pipette
    - '_first_approximation' (float): Float value containing the approximated flow rate obtained in the first step of the optimization

    ### Properties (no setter)
    - '_latest_suggestion' (DataFrame): Dataframe containing the latest feature suggestions by the BO algorithm 
    - '_latest_volume' (int): Value of the latest volume transfered during the optimization
    - '_latest_acq_value' (float): Value of the latest suggestion acquisition value
    
    ### Methods
    ## Miscelaneous 
    - set_data: Takes a dataframe calculates mean values of transfers and time to aspirate a 1000, and updates  property _data
    - update_data: Concatenates the dataframe obtained during the last measurment with the previous data, then it sets to property _data 
    - data_from_csv: Loads data from a csv file path and sets to property _data.
    - df_last_measurement: Creates a dataframe with the values from the last measurement. 
    - calibration_summary: Calculates statistics from calibration experiment
    - sigmoid: Defines sigmoid function used during mass balance controlled approximation of liquid flow in pipette tip.
    
    ## Bayesian Optimization
    - xy_split: Splits data into x and y values depending of the value of the attributes features and objectives
    - set_bounds: Set bounds of parametric space from attributes bmin and bmax
    - fit_surrogate: Fits GPR surrogate to training data
    - optimized_suggestions: Surrogate functions are sampled and likely gain is calculated by acquisition function

    ## Robotic platform control
    - cleanTip: 
    - gravimetric_transfer
    - obtainAproximateRate
    - exploreBoundaries
    - optimizeParameters
    - calibrateParameters
    """

    def __init__(self, liquid_name, density=None):
        """
        Instatiate the class
        Args:
            - liquid_nmae (str): Name of the target liquid that require liquid handling 
            optimization
            - density (float): Density of the target liquid
        """

        self.liquid_name = liquid_name
        self.density = density
        self.features = ['aspiration_rate','dispense_rate']
        self.objectives = ['%error','time_asp_1000']
        self.bmax = 1.25
        self.bmin = 0.1
        self.mean_volumes = [1000,500,300]
        self.platform = None
        self._data = pd.DataFrame(columns = ['liquid', 
                                             'pipette', 
                                             'volume', 
                                             'aspiration_rate', 
                                             'dispense_rate', 
                                             'delay_aspirate',
                                             'delay_dispense',
                                             'iteration',
                                             'liquid_level',
                                             'density', 
                                             'time', 
                                             'm', 
                                             '%error',
                                             'time_asp_1000',
                                             'acq_value'])
        self._first_approximation = None
        self._latest_acq_value = None
        self._latest_suggestion = None
        self._latest_volume = None
        self._param_dict = {
                            'aspiration_rate': None, 
                            'dispense_rate': None, 
                            'delay_aspirate': 10,  
                            'delay_dispense': 10
                            }

    
    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, df):
        self.set_data(df)
        
    @property
    def first_approximation(self):
        return self._first_approximation

    @first_approximation.setter
    def first_approximation(self,flow_rate):
        self._first_approximation = flow_rate

    @property
    def latest_acq_value(self):
        return self._latest_acq_value
    
    @property
    def latest_suggestion(self):
        return self._latest_suggestion

    @property
    def latest_volume(self):
        return self._latest_volume

    @property
    def param_dict(self):
        return self._param_dict
    
    @param_dict.setter
    def param_dict(self,dictionary):
        for key,value in dictionary.items():
            try:
                self._param_dict[key] = value
            except:
                print('parameter is not in param_dict')
    
    
    ##Miscelaneous functions

    def set_data(self,df):
        """ 
        Selects columns in a dataframe that are also in property _data, calculates iteration, 
        time to aspirate 1000 µL and mean values for transfers using the same parameters that 
        have been performed for all values in attribute mean_volumes 

        Args:
            - df (pandas.DataFrame) : Dataframe containing the data of the gravimmetric tests
            performed during an optimization
        """
       
        df = df.loc[:,self._data.columns()].copy()
        nan_columns = df.columns.to_list()
        nan_columns = [column for column in nan_columns if column not in ('liquid',
        'pipette',
        'volume',
        'aspiration_rate',
        'dispense_rate',
        'delay_aspirate',
        'delay_dispense',
        'density',
        '%error',
        'time_asp_1000',
        'acq_value',
        'iteration',
        'liquid_level')]

        df['time_asp_1000'] = 1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']

        iteration = 1
        
        if 'acq_value' not in df.columns:
            df['acq_value'] = None

        if df.loc[:,self.features].duplicated().sum()==0:
            df_mean = df
        else:
            df_duplicates = df.where(df.duplicated(self.features,keep=False)==True).dropna(how='all')
            df_incomplete = df.where(df.duplicated(self.features,keep=False)==False).dropna(how='all')
            df_mean = pd.DataFrame(columns= df.columns)
            for index,values in df_duplicates.drop_duplicates(self.features).iterrows():
                if len(df_duplicates.loc[index:index+2]) == len(self.mean_volumes):
                    mean_error =df_duplicates.loc[index:index+2,'%error'].abs().mean()
                    df_duplicates.loc[index,'%error'] = -mean_error
                    df_duplicates.loc[index, 'volume'] ='mean'+str(self.mean_volumes)
                    df_duplicates.loc[index, 'iteration']= iteration
                    df_duplicates.loc[index, 'liquid_level']= df.loc[index+2,'liquid_level']
                    df.loc[index:index+2, 'iteration'] = iteration
                    df_duplicates.loc[index, nan_columns]= 'NaN'
                    df_mean = pd.concat([df_mean,df.loc[index:index+2],df_duplicates.loc[[index]]])
                    iteration +=1 
                else:
                    df_incomplete = pd.concat([df_incomplete,df_duplicates.loc[index:index+2]]).drop_duplicates()
            df_mean = pd.concat([df_mean,df_incomplete])
            df_mean = df_mean.reset_index(drop=True)    
        self._data = df_mean
 

    def update_data(self,df):
        """ 
        Concatenates dataframe from one gravimetric test with existing data in _data and
        sets the concatenated dataframe to _data
        Args:
            - df (pandas.DataFrame) : Dataframe containing the data of one gravimetric test
        """
        self._latest_volume = df['volume'].iloc[-1]
        updated_data = pd.concat([self._data,df],ignore_index=True)
        self.set_data(updated_data)
        return self._data


    def data_from_csv(self,file_name):
        """ 
        Loads a csv file and sets to _data
        Args:
            - file_name (str) : Path of csv file
        """
        data = pd.read_csv(file_name)
        self.set_data(data)

    
    def df_last_measurement(self,error,volume= 1000):
        """ 
        Reaturns a DataFrame object containing the latest measured error for a gravimetric test
        Args:
            - error (float) : value of relative error of transfer from gravimetric test
            - volume (int) : Volume transfered in gravimetric test
        """
        self._latest_volume = volume
        last_measurement_data = self._data.iloc[[-1].copy()
        last_measurement_data.loc[updated_data.last_valid_index(),'volume'] = self._latest_volume
        last_measurement_data.loc[updated_data.last_valid_index(),'aspiration_rate']  = self._latest_suggestion['aspiration_rate']
        last_measurement_data.loc[updated_data.last_valid_index(),'dispense_rate']  = self._latest_suggestion['dispense_rate']
        last_measurement_data.loc[updated_data.last_valid_index(),'%error'] = error
        return last_measurement_data
    
    @staticmethod
    def calibration_summary(df):
        """ 
        Returns a DataFrame object containing the summary of mean transfer errors, standard deviations,
        time to transfer 1000 µL and iteration of the tested parameters in the calibration procedure
        Args:
            - df (pandas.DataFrame) : Dataframe containing values of the gravimetric tests perfored 
            during calibration test of 
        """
        if 'volume_transfered' and 'volume_error' and 'time_asp_1000' not in df.columns:
            df['volume_transfered'] = (df['m']/df['density'])*1000
            df['volume_error'] = df['volume_transfered'] - df['volume']
            df['time_asp_1000']=1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']             

        df_summary_all = pd.DataFrame()

        for volume in self.mean_volumes:
            df_experiment_v = df.where(df['volume'] == volume).dropna(how='all')
            df_summary = pd.DataFrame(columns = (f'Mean transfer volume for {volume} µL [µL]', f'Mean transfer volume error of {volume} µL [µL]', f'Mean relative error for transfer of {volume} µL [%]', f'Standard deviation for transfer of {volume} µL [µL]', f'Relative standard deviation for transfer of {volume} µL [%]') )
            data = [df_experiment_v['volume_transfered'].mean(), df_experiment_v['volume_error'].mean(), df_experiment_v['%error'].mean(), df_experiment_v['volume_transfered'].std(), (df_experiment_v['volume_transfered'].std() / df_experiment_v['volume_transfered'].mean() * 100)]
            df_summary.loc[df['liquid'].iloc[0]] = data
            df_summary_all = pd.concat([df_summary_all, df_summary], axis = 1)
        return df_summary_all 

    @staticmethod
    def sigmoid(self,x, K ,x0, B,v,A):
        """ 
        Returns a value based on the evaluation of a generalized logistic function
        Args:
            - x (float) : Value to be evaluated
            - K (float) : Upper asymptote
            - A (float) : Lower asymptote
            - x0 (float) : Starting value for a series of x
            - B (float) : Growth constant
            - v (float) : Assymetry factor
        """
        y = (K-A) / (1 + np.exp(B*(x-x0)))**(1/v) + A
        return y
    
    #BO relevant functions                            

    def xy_split(self):
        """ 
        Returns pandas.DataFrames for features (x) and objectives (y) from _data to 
        train a ML algorithm  
        """
        df_train = self._data.where(self._data['volume']=='mean'+str(self.mean_volumes)).dropna(how='all')
        x_train = df_train[self.features]
        y_train = df_train[self.objectives]
        return x_train,y_train


    def set_bounds(self, x_train):
        """
        Set the bounds for the parametric space in terms 
        of attributes bmin and bmax
        """
        return torch.vstack([x_train[0]*self.bmin, x_train[0]*self.bmax])


    def fit_surrogate(self):
        """
        Returns a GPR model for each value in attribute objectives using each value
        of attribure features, a reference point for each objective, normalized 
        training data and the bounds of the parametric space 
        """
        x_train, y_train = self.xy_split()
        x_train = torch.tensor(x_train.to_numpy(dtype=float), **tkwargs)
        y_train = torch.tensor(y_train.to_numpy(dtype=float), **tkwargs)
        y_train[:,0] = -torch.absolute(y_train[:,0])
        y_train[:,1] = -torch.absolute(y_train[:,1])

        problem_bounds = self.set_bounds(x_train)
        time_upper = 1000/problem_bounds[0][0] +1000/problem_bounds[0][1] + 10
        error_upper = y_train[:,0].abs().min()*1.25
        ref_point = torch.tensor([-error_upper,-time_upper], **tkwargs)

        train_x_gp = normalize(x_train, problem_bounds)
        models = []
        for i in range(y_train.shape[-1]):
            models.append(SingleTaskGP(train_x_gp, y_train[..., i : i + 1], outcome_transform=Standardize(m=1)))
        model1 = ModelListGP(*models)
        mll1 = SumMarginalLogLikelihood(model1.likelihood, model1)

        fit_gpytorch_model(mll1)
    
        return model1, ref_point, train_x_gp, problem_bounds
    

    def optimized_suggestions(self, random_state= 42):
        """
        Returns a dataframe witn the ten suggestions of features
        that the acquisituion function computed for most likely gain. 
        Sets _latest_acq_value and _latest_suggestion using the top sugestions
        of the dataframe
        """
        if random_state != None:
            torch.manual_seed(random_state) 
        standard_bounds = torch.zeros(2, len(self.features), **tkwargs)
        standard_bounds[1] = 1
        model1, ref_point, train_x_gp, problem_bounds = self.fit_surrogate()
        acq_func1 = qNoisyExpectedHypervolumeImprovement(
        model=model1,
        ref_point=ref_point, # for computing HV, must flip for BoTorch
        X_baseline=train_x_gp, # feed total list of train_x for this current iteration
        sampler=SobolQMCNormalSampler(sample_shape=512),  # determines how candidates are randomly proposed before selection
        objective=IdentityMCMultiOutputObjective(outcomes=np.arange(len(self.objectives)).tolist()), # optimize first n_obj col 
        prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on
        sobol1 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol2 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol_all = torch.vstack([sobol1, sobol2])
            
        acq_value_list = []
        for i in range(0, sobol_all.shape[0]):
            with torch.no_grad():
                acq_value = acq_func1(sobol_all[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())
                
        # filter the best 12 QMC candidates first
        sorted_x = sobol_all.cpu().numpy()[np.argsort((acq_value_list))]
        qnehvi_x = torch.tensor(sorted_x[-12:], **tkwargs)  
        # unormalize our training inputs back to original problem bounds
        new_x =  unnormalize(qnehvi_x, bounds=problem_bounds)
        new_x = pd.DataFrame(new_x.numpy(),columns= self.features)
        new_x['acq_value'] = sorted(acq_value_list, reverse=True)[:12]
        self._latest_suggestion = new_x[self.features].iloc[0]
        self._latest_acq_value = new_x['acq_value'].iloc[0]
        return new_x


    
    ### Methods for controlling robotic platform

    def cleanTip(self, well, repetitions =2 ):
        """
        Executes commands to clean pipette tip using 3 cycles of blowouts
        and plunger homing
        Args:
            - well (labware.well) : Well of labware where the clean tip procedure 
            will be performed
            - repetitions (int) : Number of repetitions for the clean tip procedure
        """
        
        self.platform.mover.safeMoveTo(well.top)
        
        for i in range(repetitions):
            self.platform.liquid.blowout(home=False) 
            time.sleep(5)
            self.platform.liquid.home()
            self.platform.setup.touchTip(well)
            time.sleep(5)

            self.platform.liquid.blowout(home=False) 
            time.sleep(5)
            self.platform.liquid.home()
            self.platform.setup.touchTip(well)
            time.sleep(5)

            self.platform.liquid.blowout(home=False) 
            time.sleep(5)
            self.platform.liquid.home()
            self.platform.setup.touchTip(well)
            time.sleep(5)

    
    def gravimetric_transfer(self, volume, liquid_level, source_well,balance_well):
        """
        Executes commands for one gravimetric test of a combination of liquid 
        handling parameters defined by the property param_dict. 
        Returns updated liquid level and a dataframe containing data from transfer
        Args:
            - volume (int) : Target volume for transfer
            - liquid_level (float) : Height of liquid column in source well
            - source_well (labware.well) : Well of labware where the target liquid 
            is stored
            - balance_well (labware.well) : Well of labware placed on top of automated mass balance
        """
        #liquid transfer
        #transfer start
        start = time.time() 

        #aspirate step
        self.platform.mover.safeMoveTo(source_well.from_bottom((0,0,liquid_level-5))) 
        self.platform.liquid.aspirate(volume, speed=self._param_dict['aspiration_rate'] )
        time.sleep(self._param_dict['delay_aspirate'])

        self.platform.setup.touchTip(source_well) 

        #dispense step
        self.platform.mover.safeMoveTo(balance_well.from_top((0,0,-5))) 
        self.platform.balance.tare() 
        self.platform.balance.clearCache() 
        self.platform.balance.toggleRecord(True) 
        time.sleep(5)
        self.platform.liquid.dispense(volume,speed=self._param_dict['dispense_rate'] )
        time.sleep(self._param_dict['delay_dispense'])

        #transfer termination
        finish = time.time() 
        time_m = finish - start

        self.platform.mover.safeMoveTo(source_well.top) 
        time.sleep(5)
        self.platform.balance.toggleRecord(False) 

        #do blowout
        
        self.cleanTip(source_well)

        #record transfer values 
        #calculating mass error functions
        m = (self.platform.balance.buffer_df.iloc[-10:,-1].mean()-self.platform.balance.buffer_df.iloc[:10,-1].mean())/1000 
        error = (m-self.density*volume/1000)/(self.density/1000*volume)*100
        
        #change liquid levels
        liquid_level = liquid_level - 2*m/self.density   
        
        #making new dataframe + filling it in
        df = pd.DataFrame(columns=self._data.columns)            
        
        df = pd.concat([df,pd.DataFrame({
            "liquid": self.liquid_name,
            'pipette': 'rLine1000',
            "volume": volume,
            "aspiration_rate": self._param_dict['aspiration_rate'],
            "dispense_rate": self._param_dict['dispense_rate'], 
            "delay_aspirate" : self._param_dict['delay_aspirate'],  
            "delay_dispense" : self._param_dict['delay_dispense'],
            'iteration': 'NaN',
            "liquid_level" : liquid_level,
            "density" : self.density,
            "time" : time_m,
            "m": m,
            "%error": error,
            "time_asp_1000" : 'NaN',
            "acq_value": self._latest_acq_value
            },index=[0])],ignore_index=True)
        
        return liquid_level, df

    def obtainAproximateRate(self, initial_liquid_level_balance,balance_well,speed=265, file_name=None):
        """
        Executes commands to obtain first approximation of flow rate for optimization protocol.
        Sets _first_approximation property and saves file with the recorded mass/time data
        Args:
            - initial_liquid_level_balance (float) : Height of liquid column in well on top of balance
            - balance_well (labware.well): Well of labware placed on top of automated mass balance
            - speed (float) : Speed of plunger movement defined as flow rate [µL/s] 
            - file_name (str, optional) : Path to save the recorded mass/time data
        """
        liquid_level = initial_liquid_level_balance
        
        if self.platform.liquid.isTipOn()== False:
            self.platform.setup.attachTip()
        
        self.platform.mover.safeMoveTo(balance_well.from_bottom((0,0,liquid_level-5)),descent_speed_fraction=0.25)
        #Starting balance measurement
        time.sleep(5)
        self.platform.balance.zero(wait=5)
        self.platform.balance.clearCache()
        self.platform.balance.toggleRecord(on=True)
        time.sleep(15)

        self.platform.liquid.aspirate(1000, speed=speed)

        #Switching the balance off after change in mass is less than 0.05
        while True:
            data = self.platform.balance.buffer_df
            data['Mass_smooth']= signal.savgol_filter(data['Mass'],91,1)
            data['Mass_derivative_smooth']=data['Mass_smooth'].diff()
            condition=data['Mass_derivative_smooth'].rolling(30).mean().iloc[-1]
            if condition>-0.05:
                break
        print('loop stopped')
        self.platform.balance.toggleRecord(on=False)

        self.platform.mover.moveTo(balance_well.from_top((0,0,-5)))


        #using data from balance buffer_df above, calculate time in seconds and mass derivatives
        data['ts'] = data['Time'].astype('datetime64[ns]').values.astype('float') / 10 ** 9
        data['ts']= data['ts']-data['ts'][0]
        data_fit = data.where(data['ts']>10).dropna()
        data_fit['Mass']=data_fit['Mass']-data_fit['Mass'].iloc[0]
        data_fit['Mass_smooth'] = data_fit['Mass_smooth']-data_fit['Mass_smooth'].iloc[0]

        p0 = [min(data_fit['Mass']), np.median(data_fit['ts']),1,1,max(data_fit['Mass'])+30]
        
        popt, pcov = curve_fit(self.sigmoid, data_fit['ts'], data_fit['Mass'],p0)

        mass_sigmoid = self.sigmoid(data_fit['ts'],popt[0],popt[1],popt[2],popt[3],popt[4])

        data_fit.loc[data_fit.index[0]:,'Mass_sigmoid'] = mass_sigmoid

        flow_rate = mass_sigmoid.diff()/data_fit.loc[data_fit.index[0]:,'ts'].diff()

        data_fit.loc[data_fit.index[0]:,'Flow_rate']=flow_rate

        flow_rate = mass_sigmoid.diff()/data_fit.loc[data_fit.index[0]:,'ts'].diff()

        flow_rate_max = flow_rate.min()

        flow_rate_98 = data_fit.where(data_fit['Flow_rate']<(0.05*flow_rate_max)).dropna()

        time_start, time_final = flow_rate_98.iloc[0].loc['ts'],flow_rate_98.iloc[-1].loc['ts']

        initial_flow_rate_aspirate = 1000/(time_final-time_start)
        
        self._first_approximation = initial_flow_rate_aspirate 
        
        #switching balance off and saving csv
        if file_name != None:
            data_fit.to_csv(file_name, index=False)

        self.platform.liquid.dispense(1000,speed= self._first_approximation)


    def exploreBoundaries(self, initial_liquid_level_source,source_well,balance_well):
        """
        Executes commands for 5 gravimetric test using a combination of liquid 
        handling parameters derived from the property first_approximation and 
        attributes bmin, bmax. Each combination of liquid handling paramters gravimetric 
        test is repeated for eac value of attribute mean volumes
        Args
            - initial_liquid_level_source (float) : Height of liquid column in source well
            - source_well (labware.well) : Well of labware where the target liquid 
            is stored
            - balance_well (labware.well) : Well of labware placed on top of automated mass balance
        """
        self.platform.mover.setSpeed(50)
        self.platform.mover.setHandedness(False)

        if type(self._data) == None:
            df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate', 'delay_aspirate', 'delay_dispense','iteration','liquid_level','density', 'time', 'm', '%error','time_asp_1000','acq_value'])
            df = df.astype({'liquid':str,'pipette':str})
            self.set_data(df)
        
        liquid_level = initial_liquid_level_source

        #Check if new tip is required
        if self.platform.liquid.isTipOn()== False:
            self.platform.setup.attachTip()

        volumes_list = self.mean_volumes
        
        #NOT TO BE CHANGED
        counter = 1
        iterations = 5
        #while loop
        while counter <= iterations:
            #hardcoding aspirate and dispense rates:
            if counter == 1:
                self._param_dict['aspiration_rate'] = self._first_approximation
                self._param_dict['dispense_rate'] = self._first_approximation
            if counter == 2:
                self._param_dict['aspiration_rate'] = self._first_approximation*self.bmax
                self._param_dict['dispense_rate'] = self._first_approximation*self.bmax
            if counter == 3:
                self._param_dict['aspiration_rate'] = self._first_approximation*self.bmax
                self._param_dict['dispense_rate'] = self._first_approximation*self.bmin
            if counter == 4:
                self._param_dict['aspiration_rate'] = self._first_approximation*self.bmin
                self._param_dict['dispense_rate'] = self._first_approximation*self.bmax
            if counter == 5:
                self._param_dict['aspiration_rate'] = self._first_approximation*self.bmin
                self._param_dict['dispense_rate'] = self._first_approximation*self.bmin


            #for loop
            for volume in volumes_list:
                #liquid transfer
                liquid_level,df = self.gravimetric_transfer(volume,liquid_level,source_well,balance_well)
                
                m=df.m.iloc[-1]

                self.set_data(pd.concat([self._data,df]).reset_index(drop=True))
                #printing checks
                print("LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/self.density) + "   ITERATION: " + str(counter) + ", " + "VOLUME: " + str(volume))    

                #liquid level checks
                if (1.2*m/self.density > 1.2) or (1.2*m/self.density < 0):
                    break
                if (liquid_level > initial_liquid_level_source) or (liquid_level < 6):
                    break

            counter += 1


    def optimizeParameters(self, initial_liquid_level_source,source_well,balance_well,iterations=5,file_name=None):
        """
        Executes commands for n iterations of gravimetric test using a combination of liquid 
        handling parameters suggested by a BO algorithm. Each test is repeated for each
        value of attribute mean volumes.
        Args
            - initial_liquid_level_source (float) : Height of liquid column in source well
            - source_well (labware.well) : Well of labware where the target liquid 
            is stored
            - balance_well (labware.well) : Well of labware placed on top of automated mass balance
            - iterations (int) : Number of optimization iterations
            - file_name (str, optional) : Path to save the values recorded in property data during the optimization
        """
        self.platform.mover.setSpeed(50)
        self.platform.mover.setHandedness(False)

        liquid_level = initial_liquid_level_source

        #Check if new tip is required
        if self.platform.liquid.isTipOn()== False:
            self.platform.setup.attachTip()

        volumes_list = self.mean_volumes
        
        #NOT TO BE CHANGED
        counter = 1
       
        #while loop
        while counter <= iterations:
            #getting botorch suggestions + implementing it in liquids_dict
            self.optimized_suggestions()

            self._param_dict['aspiration_rate'] = self._latest_suggestion['aspiration_rate']
            self._param_dict['dispense_rate'] = self._latest_suggestion['dispense_rate']
            #for loop
            for volume in volumes_list:
                #liquid transfer
                liquid_level,df = self.gravimetric_transfer(volume,liquid_level,source_well,balance_well)
                
                m=df.m.iloc[-1]

                self.set_data(pd.concat([self._data,df]).reset_index(drop=True))
                #printing checks
                print("LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/self.density) + "   ITERATION: " + str(counter) + ", " + "VOLUME: " + str(volume))    


                
                #printing checks
                print("LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/self.density) + "   ITERATION: " + str(counter) + ", " + "VOLUME: " + str(volume))    

                #liquid level checks
                if (1.2*m/self.density > 1.2) or (1.2*m/self.density < 0):
                    break
                if (liquid_level > initial_liquid_level_source) or (liquid_level < 6):
                    break
            
            counter += 1
        if file_name != None:
            self._data.to_csv(file_name, index=False)


    def calibrateParameters(self, initial_liquid_level_source,source_well,balance_well,iterations=10, file_name=None):
        """
        Executes commands for n iterations of gravimetric test using the best recorded 
        combination of liquid handling parameters in property data. 
        The test is performed n times for each value of attribute mean volumes.
        Args
            - initial_liquid_level_source (float) : Height of liquid column in source well
            - source_well (labware.well) : Well of labware where the target liquid 
            is stored
            - balance_well (labware.well) : Well of labware placed on top of automated mass balance
            - iterations (int) : Number of transfer per volume calibrated
            - file_name (str, optional) : Path to save csv with the values recorded during calibration 
            and the statistical summary
        """

        self.platform.mover.setSpeed(50)
        self.platform.mover.setHandedness(False)

        liquid_level = initial_liquid_level_source

        # Check if new tip is required
        if self.platform.liquid.isTipOn()== False:
            self.platform.setup.attachTip()

        volumes_list = self.mean_volumes
        
        #NOT TO BE CHANGED
     
        
        mean_average_data = self._data.where(self._data.volume == 'mean'+str(self.mean_volumes))
        mean_average_data = mean_average_data.where(mean_average_data.iteration>5).dropna()
        best_parameter_index = mean_average_data[mean_average_data['%error']==mean_average_data['%error'].max()].index

        self._param_dict['aspiration_rate'] = self._data.loc[best_parameter_index,'aspiration_rate'].values[0]
        self._param_dict['dispense_rate'] = self._data.loc[best_parameter_index,'dispense_rate'].values[0]
        
        calibration_df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate', 'delay_aspirate', 'delay_dispense','liquid_level','density', 'm', '%error'])
        
        #for loop
            
        for volume in volumes_list:
            counter = 1
            # #while loop
            while counter <= iterations:
             
                #liquid transfer
                liquid_level,df = self.gravimetric_transfer(volume,liquid_level,source_well,balance_well)
                
                calibration_df = pd.concat([calibration_df,df[calibration_df.columns]]).reset_index(drop=True)

                m=df.m.iloc[-1]

                #printing checks
                print("Mass: "+str(m)+"LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/self.density) + "   ITERATION: " + str(counter) + ", " + "VOLUME: " + str(volume))    

                #liquid level checks
                if (1.2*m/self.density > 1.2) or (1.2*m/self.density < 0):
                    break
                if (liquid_level > initial_liquid_level_source) or (liquid_level < 6):
                    break

                counter += 1
            #liquid level checks
            if (1.2*m/self.density > 1.2) or (1.2*m/self.density < 0):
                break
            if (liquid_level > initial_liquid_level_source) or (liquid_level < 6): 
                break
        
        calibration_df['volume_transfered'] = calibration_df['m']/calibration_df['density']*1000
        calibration_df['volume_error'] = calibration_df['volume_transfered'] - calibration_df['volume']
        calibration_df['time_asp_1000'] = 1000/calibration_df['aspiration_rate'] + 1000/calibration_df['dispense_rate'] + calibration_df['delay_aspirate'] + calibration_df['delay_dispense']       
        
        calibration_summary_df= self.calibration_summary(calibration_df)

        if file_name != None:
            calibration_df.to_csv(file_name, index=False)
            calibration_summary_df.to_csv(file_name[:-4]+'_summary.csv')
        


# 3. Semi-automated implemntation

The following cells serve as an example of how to implement a semi-automated MOBO of liquid transfer paramters using the BO_LiqTransfer class. This implementation is run in parallel with the script that is controlling the liquid transfer robot. In **publication** this was used in parallel with a jupyternotbook controlling a Opentrons OT2 robot (**script link**).

The process for the MOBO is as follwos

1. Create BO_LiqTransfer object and load initial trasnfer data set. The initial transfer data set should be previously acquired through gravimmetric testing of several combination of liquid handling parameters

2. Run optimized_suggestions() method to obtain suggested liquid handling paramters by BO algorithm. This function first trains surrogate models that predict the optimization objective (default: relative error and time to aspirate 1000 µL) from predifined liquid handling parameter features (default: aspiration and dispense rates). After an acquisition function will be used to suggest new combination of liquid t ransfer paramters taht will likely minimize the objectives. 

    After, input the optimized suggestions in the script controlling the liquid handling robot and perform gravimmetric test.

3. Update the the data with each volume tested

4. Iterate steps 2 and 3 until optimal solutions are found. 


### 1. Create BO_LiqTransfer object and load initial trasnfer data set.

Please set liquid name and volume to transfer according to the experiment and load initial transfer data

In [ ]:
from BO_liquid_transfer import BO_LiqTransfer       # shared class

# Change according to experiment
liquid_name = 'Viscosity_std_1275' 

# Do not change
liq = BO_LiqTransfer(liquid_name, pipette_brand='ot2')
liq.data_from_csv('')
liq._data


### 2.  Run optimized_suggestions() method to obtain suggested liquid handling paramters by BO algorithm.



In [ ]:
liq.optimized_suggestions()

### 3. Update the the data with each volume tested

In [ ]:
volume= 300
last_measurement_data = liq.df_last_measurement(-0.840723, volume)
liq.update_data(last_measurement_data)

In [ ]:
#save after each standard-experiment iteration
liq._data.to_csv('C:\\Users\\amdm_\\OneDrive\\Documents\\GitHub\\viscosity_liquid_transfer_Pablo\\Opentrons_experiments\\BOTorch_optimization\\VS_code_csv\\'+liquid_name+'_'+'duplicate_unused_exp3.csv', index = False)

### 4. Iterate steps 2 and 3 until optimal solutions are found. 

## 4. Fully-automated implementation

The following cells serve as an example of how to implement a semi-automated MOBO of liquid transfer paramters using the BO_LiqTransfer class. In this implementation the initial approximation of the flow rate within the pipette tip is calculated using human vision.  In **publication** this was used to perform the multi-objective optimization of liquid handling parameters for the transport of viscous liquids with a rLine1000 automated pipette.


The process for the MOBO is as follwos

1. Initialize robotic platform

2. Create BO_LiqTransfer object by inputting liquid name and density. 

3. Obtain approximate flow rate

4. Run exploreBoundaries(). To obtain intial data set that will be used for the optimization protocol,

5. Run optimizeParameters()

6. Run calibrateParameters()

### 1. Initialize robotic platform


For this example we use control-lab-ly python package to control an automated platform that consists of a autoamted mass balance, a Sartorious rLine1000 pipette, M1 DOBOT scara arm and a deck object cotnainig the location of the labware used in the platform.


A yaml file that defines all the automated equipment present in the platform is used to initialize and connect with the hardware, using the load_setup. This function returns an object that can be used to point to the objects that control each automated equipment. The objects controlling the hardware can be accesed from the *platform* variable as follows:

- platform.setup: This variable points to the object controlling the robotic arm and autoamted pipette. This variable is used to execute the commands that require both a pipette and a robot arm (i.e. picking up a tip). It also can be used to point to the objects that control the independent functions of the robotic arm and the pipette using the following vairables:

    - platform.setup.mover: Variable pointing to the object that controls exclusively the robotic arm
    - platform.setup.liquid: Object that controls exlusively the automated pipette

- platform.balance: This variable points to the object controlling the automated mass balance



In [ ]:
#Import robot related packages and run setup
from pathlib import Path
from controllably import load_setup     # pip install control-lab-ly

HERE = str(Path().parent.absolute()).replace('\\', '/')


platform = load_setup(config_file=f'{HERE}/config.yaml') # initialize objects to control automated setup
platform.setup.mover.verbose = False 


    




The compound object *platform.setup* can also hold the positional inforamtion of the labware placed in the deck of the platform by loading a json file that defines the coordinate position of each labware slot and the path to the json file containing the inforamtion of the spatial distribution of the "wells" of each of the labware. This operation is similar to laoding labware into OT2 decks for reference ().

In [ ]:
from controllably import load_deck

load_deck(device=platform.setup, layout_file='layout.json') 

balance_deck = platform.setup.deck.slots['1'] #Variable that holds the positional information of the balance within the cartesian space of the deck 
source = platform.setup.deck.slots['2'] #Variable that holds the positional information of the labware containing the source of the test liquid within the cartesian space of the deck 


### 2. Create BO_LiqTransfer

To intizlize the BO_LiqTransfer object pass the follwing arguments:
- liquid name : String that identifies the liquid that requires liquid handling paramter optimization 
- density : Value of the density of the target liquid in g/mL, this value will be required to calculate the transfer error during the gravimmetric testing
- platform : Variable that points to all the automated objects of the automated platform. This variable is required for the autoamted optimization of the liquid handling paramters using the methods defined in the BO_LiqTransfer() class

In [ ]:
liq = BO_LiqTransfer(liquid_name = 'Viscous_std_204',density = 0.8736, platform = platform) 

### 3. Obtain target liquid approximate flow rate
This cotnains the code to obtain an approximate flow rate value of the target viscous liquid within the pipette tip. the first cell executes an aspiration action at 5 mm below the surface of the viscous liquid. As the aspiration starts a timestamp is taken. The user will observe the upward movement of the visoucs liquid into the pipette tip. Once the user has determined that the movement stopped, it runs the second cell where a second stamp time is taken. Then the time to aspirate 1000 µL is calculated and used to obtain an approximate flow rate. Finally the liquid will be returned to the source vial and a clean tip procedure will be performed to remove any excess liquid. 

In [ ]:
#This commands will aspirate 1000ul liquid at standard flow_rate.aspirate of pippette. A timer well be started just before aspiration starts
liquid_level = 50

platform.setup.mover.safeMoveTo(source.wells['A1'].from_bottom((0,0,liquid_level-5)))
start = time.time()
platform.setup.mover.liquid.aspirate(1000)

In [ ]:
#Run this cell when no further flow of liquid into the pipette tip is observed. Calculates an approximate flow rate for 
#aspiration
finish = time.time()
t_aspirate = finish-start
flow_rate_aspirate = 1000/t_aspirate
flow_rate_aspirate

liq.first_approximation = flow_rate_aspirate


platform.setup.mover.safeMoveTo(source.wells['A1'].top)
platform.setup.liquid.dispense(1000, speed = round(flow_rate_aspirate,flow_rate_aspirate))

liq.cleanTip(well=source.wells['A1'])


### 4. Run exploreBoundaries()

This method contains the code that executes the actions to perform the gravimetric testing of liquid handling paramters that are found at the boudnaries of the parametric space (i.e. values of maximum and minimmum aspiration rate expected). The method required the following arguments to be pased:
- initial_liquid_level_source : Initial height of the column of liquid in the vial that will be used as a source to draw for the transfer procedures.  
- source_well : Object thaht contains the postion of the vial in the platformd deck.
- balance_well : Object that contains the postion of the vial on the balance in the platformd deck.
- file_name : File name to save dataframe as a csv file contianing the data form the gravimmetric testing.


The code perfomrs the gravimetric test for the tranfer of the target volumes using the following aspiration and dispense rates:
1.  aspiration rate = Approximated flow rate , dispense rate = Approximated flow rate
2.  aspiration rate = liq.bmax x Approximated flow rate , dispense rate = liq.bmax x Approximated flow rate
3.  aspiration rate = liq.bmax x Approximated flow rate , dispense rate = liq.bmin x Approximated flow rate
4.  aspiration rate = liq.bmin x Approximated flow rate , dispense rate = liq.bmax Approximated flow rate
5.  aspiration rate = liq.bmin x Approximated flow rate , dispense rate = liq.bmin x Approximated flow rate

Where liq.max = 1.25 and liq.min = 0.1 by default.

The data gatehered during this experiments will be used to train the initial GPR for the MOBO. 


In [ ]:
liq.exploreBoundaries(initial_liquid_level_source=42, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'])

source_iquid_level = liq._data['liquid_level'].iloc[-1]

### 5. Run optimizeParameters()
This method contains the code that executes the actions to perform the gravimetric testing of liquid handling paramters suggeted by the MOBO algorithm. The method required the following arguments to be pased:
- initial_liquid_level_source : Initial height of the column of liquid in the vial that will be used as a source to draw for the transfer procedures.  
- source_well : Object thaht contains the postion of the vial in the platformd deck.
- balance_well : Object that contains the postion of the vial on the balance in the platformd deck.
- iterations : Number of optimization iterations 
- file_name : File name to save dataframe as a csv file contianing the data form the gravimmetric testing.



In [ ]:
liq.optimize_parameters(initial_liquid_level_source = source_iquid_level, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'], iterations=5, file_name = 'Viscosity_std_204.csv')


### 6. Run calibrateParameters()

This method contains the code that executes the actions to perform 10 gravimetric tests per target volumes using the best liquid handling parameter combination found during the optimization step. This method requires the following arguments to be passed:

- initial_liquid_level_source : Initial height of the column of liquid in the vial that will be used as a source to draw for the transfer procedures.  
- source_well : Object thaht contains the postion of the vial in the platformd deck.
- balance_well : Object that contains the postion of the vial on the balance in the platformd deck. 
- file_name : File name to save a dataframe as a csv file contianing the data form the gravimmetric testing and second file containing the summary of the statistics of the mass transfer.



In [ ]:
liq.calibrate_parameters(46.5,source_well=source.wells['A1'],balance_well=balance_deck.wells['A1'],file_name='Viscosity_std_204_calibration.csv')

## 4. Fully-automated implementation

The following cells serve as an example of how to implement a fully-automated MOBO of liquid transfer paramters using the BO_LiqTransfer class. This implementation is run in parallel with the script that is controlling the liquid transfer robot. In **publication** this was used to perform the multi-objective optimization of liquid handling parameters for the transport of viscous liquids with minimal human input

The process for the MOBO is as follwos

1. Initialize robotic platform
2. Create BO_LiqTransfer object by inputting liqudi name and density. 

3. Run obtainAproximateRate(). To calculate initial flow rate to be tested for the optimization protocol


4. Run exploreBoundaries(). To obtain intial data set that will be used for the optimization protocol,

5. Run optimizeParameters()

6. Run calibrateParameters()

Step 3 is the only difference from section 4, thus the explanation for those sections are not repeated.

### 1. Initialize robotic platform

In [ ]:
#Import robot related packages and run setup
from pathlib import Path
from controllably import load_setup     # pip install control-lab-ly

HERE = str(Path().parent.absolute()).replace('\\', '/')


platform = load_setup(config_file=f'{HERE}/config.yaml') # initialize objects to control automated setup
platform.setup.mover.verbose = False 


    
from controllably import load_deck

load_deck(device=platform.setup, layout_file='layout.json') 

balance_deck = platform.setup.deck.slots['1'] #Variable that holds the positional information of the balance within the cartesian space of the deck 
source = platform.setup.deck.slots['2'] #Variable that holds the positional information of the labware containing the source of the test liquid within the cartesian space of the deck 



### 2. Create BO_LiqTransfer


In [ ]:
liq = BO_LiqTransfer(liquid_name = 'Viscous_std_204',density = 0.8736, platform = platform) 

### 3. Run obtainAproximateRate()

This method contains the code that executes the actions required to obtain an approximated value of the flow rate within the pipette tip during the aspiration of the target liquid. This value will be used as an starting point for the MOBO of the liqudi handling paramters. The method required the following arguments to be pased:
- initial_liquid_level_balance : Initial height of the column of liquid in the vial is located on the balance.  
- balance_well : Object that contains the postion of the vial on the balance in the platformd deck..
- file_name : File name to save dataframe as a csv file contianing the data form the change in mass of the vial during the estimation of the approximate flow rate.


For further information refer to the protocol described in **publication**. 

In [ ]:
liq.obtainAproximateRate(initial_liquid_level_balance=7.5, balance_well= balance_deck.wells['A1'],file_name='BPAEDMA_flow_rate.csv')
liq.cleanTip(well=source.wells['A1'])

### 4. Run exploreBoundaries()

In [ ]:
liq.exploreBoundaries(initial_liquid_level_source=42, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'])

source_iquid_level = liq._data['liquid_level'].iloc[-1]

### 5. Run optimizeParameters()



In [ ]:
liq.optimize_parameters(initial_liquid_level_source = source_iquid_level, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'], iterations=5, file_name = 'Viscosity_std_204.csv')


### 6. Run calibrateParameters()

In [ ]:
liq.calibrate_parameters(46.5,source_well=source.wells['A1'],balance_well=balance_deck.wells['A1'],file_name='Viscosity_std_204_calibration.csv')